In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import optimizers
import numpy as np
from keras.layers.core import Lambda
from keras import backend as K
from keras import regularizers
from tensorflow.python.keras.callbacks import TensorBoard
import time
import datetime
import os

In [3]:
tensor_board_VGG = TensorBoard(log_dir = os.path.join("logs","VGG"
                                                  "fit",
                                                  datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),
                                                 ))

In [4]:


class cifar10vgg:
    def __init__(self,train=True):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('cifar10vgg.h5')


    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model


    def normalize(self,X_train,X_test):
        #this function normalize inputs for zero mean and unit variance
        # it is used when training a model.
        # Input: training set and test set
        # Output: normalized training set and test set according to the trianing set statistics.
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        #this function is used to normalize instances in production according to saved training set statistics
        # Input: X - a training set
        # Output X - a normalized training set according to normalization constants.

        #these values produced during first training and are general for the standard cifar10 training set normalization
        mean = 120.707
        std = 64.15
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x,batch_size)

    def train(self,model):

        #training parameters
        batch_size = 128
        maxepoches = 250
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20
        # The data, shuffled and split between train and test sets:
        (x_train, y_train), (x_test, y_test) = cifar10.load_data()
        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train, x_test = self.normalize(x_train, x_test)

        y_train = keras.utils.to_categorical(y_train, self.num_classes)
        y_test = keras.utils.to_categorical(y_test, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)



        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        historytemp = model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test, y_test),callbacks=[reduce_lr,tensor_board_VGG],verbose=2)
        model.save_weights('cifar10vgg.h5')
        return model

if __name__ == '__main__':


    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)

    model = cifar10vgg()

    predicted_x = model.predict(x_test)
    residuals = np.argmax(predicted_x,1)!=np.argmax(y_test,1)

    loss = sum(residuals)/len(residuals)
    print("the validation 0/1 loss is: ",loss)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 390 steps, validate on 10000 samples
Epoch 1/250
390/390 - 61s - loss: 20.2758 - accuracy: 0.1940 - val_loss: 16.1584 - val_accuracy: 0.1018
Epoch 2/250
390/390 - 49s - loss: 10.5331 - accuracy: 0.2957 - val_loss: 9.1758 - val_accuracy: 0.1093
Epoch 3/250
390/390 - 49s - loss: 5.7154 - accuracy: 0.3735 - val_loss: 5.4121 - val_accuracy: 0.1305
Epoch 4/250
390/390 - 49s - loss: 3.4630 - accuracy: 0.4651 - val_loss: 3.2906 - val_accuracy: 0.3744
Epoch 5/250
390/390 - 49s - loss: 2.3981 - accuracy: 0.5392 - val_loss: 2.3613 - val_accuracy: 0.4971
Epoch 6/250
390/390 - 49s - loss: 1.8856 - accuracy: 0.6010 - val_loss: 1.9193 - val_accuracy: 0.5605
Epoch 7/250
390/390 - 49s - loss: 1.6584 - accuracy: 0.6369 - val_loss: 1.6971 - val_accuracy: 0.6300
Epoch 8/250
390/390 - 49s - loss: 1.5709 - accuracy: 0.6623 - val_loss: 1.4252 - val_accuracy: 0.7110
Epoch 9/250
390/390 - 49s - loss:

Epoch 77/250
390/390 - 49s - loss: 0.7313 - accuracy: 0.8902 - val_loss: 0.7144 - val_accuracy: 0.8968
Epoch 78/250
390/390 - 50s - loss: 0.7281 - accuracy: 0.8917 - val_loss: 0.7711 - val_accuracy: 0.8807
Epoch 79/250
390/390 - 49s - loss: 0.7259 - accuracy: 0.8911 - val_loss: 0.7121 - val_accuracy: 0.8987
Epoch 80/250
390/390 - 49s - loss: 0.7259 - accuracy: 0.8924 - val_loss: 0.6995 - val_accuracy: 0.9023
Epoch 81/250
390/390 - 49s - loss: 0.6658 - accuracy: 0.9111 - val_loss: 0.6551 - val_accuracy: 0.9141
Epoch 82/250
390/390 - 49s - loss: 0.6363 - accuracy: 0.9179 - val_loss: 0.6549 - val_accuracy: 0.9120
Epoch 83/250
390/390 - 49s - loss: 0.6168 - accuracy: 0.9214 - val_loss: 0.6611 - val_accuracy: 0.9079
Epoch 84/250
390/390 - 49s - loss: 0.6074 - accuracy: 0.9218 - val_loss: 0.6546 - val_accuracy: 0.9105
Epoch 85/250
390/390 - 49s - loss: 0.5997 - accuracy: 0.9219 - val_loss: 0.6263 - val_accuracy: 0.9165
Epoch 86/250
390/390 - 49s - loss: 0.5867 - accuracy: 0.9233 - val_loss: 

Epoch 156/250
390/390 - 49s - loss: 0.2854 - accuracy: 0.9776 - val_loss: 0.4837 - val_accuracy: 0.9302
Epoch 157/250
390/390 - 49s - loss: 0.2816 - accuracy: 0.9775 - val_loss: 0.4799 - val_accuracy: 0.9330
Epoch 158/250
390/390 - 49s - loss: 0.2861 - accuracy: 0.9760 - val_loss: 0.4892 - val_accuracy: 0.9287
Epoch 159/250
390/390 - 49s - loss: 0.2800 - accuracy: 0.9784 - val_loss: 0.4877 - val_accuracy: 0.9291
Epoch 160/250
390/390 - 49s - loss: 0.2783 - accuracy: 0.9779 - val_loss: 0.4851 - val_accuracy: 0.9302
Epoch 161/250
390/390 - 49s - loss: 0.2755 - accuracy: 0.9793 - val_loss: 0.4877 - val_accuracy: 0.9290
Epoch 162/250
390/390 - 49s - loss: 0.2722 - accuracy: 0.9803 - val_loss: 0.4858 - val_accuracy: 0.9298
Epoch 163/250
390/390 - 49s - loss: 0.2681 - accuracy: 0.9814 - val_loss: 0.4846 - val_accuracy: 0.9305
Epoch 164/250
390/390 - 49s - loss: 0.2673 - accuracy: 0.9808 - val_loss: 0.4838 - val_accuracy: 0.9305
Epoch 165/250
390/390 - 49s - loss: 0.2668 - accuracy: 0.9815 - 

Epoch 235/250
390/390 - 49s - loss: 0.2342 - accuracy: 0.9869 - val_loss: 0.4767 - val_accuracy: 0.9329
Epoch 236/250
390/390 - 49s - loss: 0.2327 - accuracy: 0.9871 - val_loss: 0.4750 - val_accuracy: 0.9323
Epoch 237/250
390/390 - 49s - loss: 0.2331 - accuracy: 0.9870 - val_loss: 0.4742 - val_accuracy: 0.9330
Epoch 238/250
390/390 - 49s - loss: 0.2358 - accuracy: 0.9864 - val_loss: 0.4748 - val_accuracy: 0.9329
Epoch 239/250
390/390 - 49s - loss: 0.2324 - accuracy: 0.9874 - val_loss: 0.4751 - val_accuracy: 0.9325
Epoch 240/250
390/390 - 49s - loss: 0.2338 - accuracy: 0.9868 - val_loss: 0.4764 - val_accuracy: 0.9321
Epoch 241/250
390/390 - 49s - loss: 0.2330 - accuracy: 0.9871 - val_loss: 0.4750 - val_accuracy: 0.9324
Epoch 242/250
390/390 - 49s - loss: 0.2333 - accuracy: 0.9866 - val_loss: 0.4738 - val_accuracy: 0.9329
Epoch 243/250
390/390 - 49s - loss: 0.2318 - accuracy: 0.9876 - val_loss: 0.4729 - val_accuracy: 0.9332
Epoch 244/250
390/390 - 49s - loss: 0.2319 - accuracy: 0.9872 - 